## ASVP-ESD 데이터셋을 가져와보자.

데이터셋은 아래 홈페이지에서 받을 수 있다.

https://github.com/HLTSingapore/Emotional-Speech-Data

문서의 설명으로부터 파일 이름에 따른 정보를 얻을 수 있었다. 이를 기반으로 아이의 울음소리에 대한 음성만을 추출한 뒤 csv 파일을 통해 정보를 저장하자.

아래는 파일 이름에 따른 정보 예시이다.

**03-01-06-01-02-12-02-01-01-16.wav**

1. 오디오 전용 (03)

2. 음향기기 사용 (01)

3. 두려움 (06)

4. 강한 목소리 (01)

5. Statement (02)

6. 남자(짝수는 남, 홀수는 여)

7. 나이는 20세에서 64세 사이

8. 출처-웹사이트

9. 언어-중국어

<br>

3번인 감정, 6번인 성별 또한 추출할 것이다. 감정과 성별에 대한 정보는 아래와 같다.

4세 이하의 나이는 구별하고 있지 않다.

<br>

**감정**

- 01 = 지루함, 한숨

- 02 = 중립, 진정

- 03 = 행복, 웃음

- 04 = 슬프다, 울다

- 05 = 분노, 투덜거림, 좌절

- 06 = 두렵다, 비명을 지르다, 당황하다

- 07 = 혐오, 혐오, 경멸

- 08 = 놀라다, 숨이 막히다, 놀라다

- 09 = 흥분

- 10 = 즐거움

- 11 = 고통, 신음

- 12 = 실망, 반대

- 13 = 호흡

<br>

**성별**

짝수는 남성, 홀수는 여성으로 구분한다. 숫자의 의미는 명시되어 있지 않다.


## 주의!

Audio 음성파일에서 baby에 해당하는 음성들을 들어보았는데 대부분 완전한 아이에 대한 내용이 아닌 것 같다. 측정한 기준이 다른 것으로 예상된다.

따라서 아이의 음성만 모아둔 Bonus 부분의 음성만 사용하자.


In [63]:
# Load packages
import os
import numpy as np
import pandas as pd
import seaborn as sns
from shutil import move
import matplotlib.pyplot as plt

In [64]:
# OS setup
main_path = os.getcwd()
data_path = os.path.join(main_path, 'data')
origin_audio_path = '/Users/jaewone/Downloads/ASVP-ESD-Update/Bonus'
csv_path = os.path.join(main_path, 'info.csv')

In [65]:
# Get baby audio file path list
baby_audio_list = []

for folder in os.listdir(origin_audio_path):
    for file in os.listdir(os.path.join(origin_audio_path, folder)):
        # check file is wav file
        file_name, format = file.rsplit('.', 1)
        if not format == 'wav':
            break

        # Append path if audio is sound of baby
        info = file_name.split('-')
        baby_audio_list.append(
            [info[2], info[5], '-'.join(info[9:]), os.path.join(origin_audio_path, folder, file)])

In [66]:
# trans data

def get_feel(feel_code):
    if feel_code == '01':
        return 'bored'
    elif feel_code == '03':
        return 'happy'
    elif feel_code == '04':
        return 'sad'
    elif feel_code == '06':
        return 'fearful'
    elif feel_code == '11':
        return 'pain'
    elif feel_code == '17':
        return 'disgust'
    elif feel_code == '02':
        return 'calm'
    return 'calm'


# 감정은 아래 7종류만 존재한다: {'11', '17', '01', '03', '02', '04', '06'}
print(set([info[0] for info in baby_audio_list]))

feel_info = [get_feel(info[0]) for info in baby_audio_list]
gender_info = ['m' if int(info[1]) %
               2 == 0 else 'f' for info in baby_audio_list]
detail_info = [''.join(info[2][:5]) for info in baby_audio_list]

{'17', '11', '02', '03', '01', '06', '04'}


In [67]:
# Load csv
origin_df = pd.read_csv(csv_path, index_col=0)
origin_df.tail(3)

,state,gender,age,source,file,detail
454,hungry,m,4,donate_a_cry,F24DE44B-762C-4149-AC92-96A5E57ED118-143081691...,NaN
455,hungry,m,72,donate_a_cry,bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-143063764...,NaN
456,hungry,f,26,donate_a_cry,ca7d8d54-13cb-4060-8db4-90651b60c078-143467476...,NaN


In [68]:
# trans to dataframe
df = pd.DataFrame(list(zip(feel_info, gender_info, detail_info, [
                  info[3] for info in baby_audio_list])), columns=['state', 'gender', 'detail', 'file'])
df.tail(3)

,state,gender,detail,file
1336,sad,m,34,/Users/jaewone/Downloads/ASVP-ESD-Update/Bonus...
1337,sad,m,34,/Users/jaewone/Downloads/ASVP-ESD-Update/Bonus...
1338,calm,m,12,/Users/jaewone/Downloads/ASVP-ESD-Update/Bonus...


In [69]:
# move file to data folder
def move_all_file_in_folder(from_path, to_path, extension=None):
  for (path, dirs, files) in os.walk(from_path):
    for dir in dirs:
      move_all_file_in_folder(os.path.join(from_path, dir), to_path, extension)
    for file in files:
      if extension:
        s = file.rsplit('.', 1)
        if len(s) == 1 or f'.{s[1]}' not in extension:
          continue
      move(os.path.join(path, file), os.path.join(to_path, file))

move_all_file_in_folder(origin_audio_path, data_path, extension=['.wav'])

In [70]:
# Set detail feel and age
def get_detail_feel(feel):
    if not type(feel) == str:
        return ''
    
    if(feel == '77'):
        return 'high_noise'
    if feel == '66':
        return 'max-sound'
    
    try:
        num = int(feel)
    except:
        return ''
    
    if num < 10:
        return ''
    
    state_num = num % 10
    if state_num == 4:
        return 'sad'
    if state_num == 5:
        return 'happy'
    if state_num == 6:
        return 'fearful'
    if state_num == 7:
        return 'disgust'
    if state_num == 8:
        return 'surprise'
    return ''

def combin_row(df):
    a = df[0]
    b = df[1]
    if a == '' and b == '':
        return ''
    if b == '':
        return a
    if a == '':
        return b
    return f'{a}/{b}'

detail_df = (pd.Series(df.detail.values)
               .str.split('-', expand=True)
               .applymap(get_detail_feel)
               .apply(lambda row: combin_row(row), axis=1)
               )

df['detail'] = detail_df
df['file'] = df['file'].str.replace(origin_audio_path, '').str.replace(
    '|'.join([f'/actor_{folder_name}/' for folder_name in ['50', '150', '200']]), '')

df['age'] = ''
df['source'] = 'asvp-esd'
df.tail(3)

/var/folders/10/9_p98m6j42n84y4wmf_k0krc0000gn/T/ipykernel_63180/317581192.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  df['file'] = df['file'].str.replace(origin_audio_path, '').str.replace(


,state,gender,detail,file,age,source
1336,sad,m,sad,03-02-04-01-06-200-05-02-01-34.wav,,asvp-esd
1337,sad,m,sad,03-01-04-01-01-200-02-02-01-34.wav,,asvp-esd
1338,calm,m,,03-02-02-01-02-200-05-02-01-12.wav,,asvp-esd


In [71]:
origin_df = pd.read_csv(csv_path, index_col=0)
origin_df = origin_df.fillna('')
origin_df

,state,gender,age,source,file,detail
0,belly_pain,m,26,donate_a_cry,999bf14b-e417-4b44-b746-9253f81efe38-143097400...,
1,belly_pain,f,26,donate_a_cry,C421C6FE-DFEE-4080-8AEA-848E7CE4756B-143054833...,
2,belly_pain,m,4,donate_a_cry,d6cda191-4962-4308-9a36-46d5648a95ed-143115926...,
3,belly_pain,m,72,donate_a_cry,643D64AD-B711-469A-AF69-55C0D5D3E30F-143013852...,
4,belly_pain,m,72,donate_a_cry,643D64AD-B711-469A-AF69-55C0D5D3E30F-143013864...,
...,...,...,...,...,...,...
452,hungry,f,26,donate_a_cry,7A864194-4B6B-4850-ABBF-0C2A8170107B-143684679...,
453,hungry,f,48,donate_a_cry,C421C6FE-DFEE-4080-8AEA-848E7CE4756B-143054825...,
454,hungry,m,4,donate_a_cry,F24DE44B-762C-4149-AC92-96A5E57ED118-143081691...,
455,hungry,m,72,donate_a_cry,bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-143063764...,


In [72]:
# append to csv
origin_df = pd.read_csv(csv_path, index_col=0)
origin_df = pd.concat([origin_df, df])
origin_df = origin_df.reset_index(drop=True)

# Update csv
origin_df.to_csv(csv_path)
origin_df.tail(3)

,state,gender,age,source,file,detail
1793,sad,m,,asvp-esd,03-02-04-01-06-200-05-02-01-34.wav,sad
1794,sad,m,,asvp-esd,03-01-04-01-01-200-02-02-01-34.wav,sad
1795,calm,m,,asvp-esd,03-02-02-01-02-200-05-02-01-12.wav,
